# Ground States: Imaginary Time Evolution
## Category: Prerequsites
## Prerequisites: Quantum Harmonic Oscillator, Time Evolution: Split Operator, Quantum Mechanics
## Author: Christina Lee

$$
\Psi (t) = \sum_i a_i(t) \phi_i = \sum_i a_i (0) e^{-i E_i t} \phi_i
$$

$$
\Psi (-it) = \sum_i a_i (0) e^{- E_i t} \phi_i
$$

In this equation, we can se that all eigenstates dissapate away in an imaginary time system, but the states with higher energy will dissipate more rapidly.  Over time, the lowest energy state will come to dominate the wavefunction.  We do this by performing a [Wick rotation](https://en.wikipedia.org/wiki/Wick_rotation) on time $(t \mapsto -i t)$, taking it into the complex domain, as shown above.



In [ ]:
using FFTW
using Plots
gr()

In [ ]:
# Set Time Parameters
t0=0
tf=5
dt=.01

# Set Space Grid Parameters 
dx=.05
xmax=6
# xmin will be -xmax.  Making the situation symmetric


In [ ]:
t=collect(t0:dt:tf)
x=collect(-xmax:dx:xmax)

nt=length(t)
N=length(x)

dk=2*π/(N*dx)
k = [ collect(0:((N-1)/2)) ; collect(-(N-1)/2:-1) ] *2*π/(N*dx);

In the next cell, we define our evolution operators.  

In [ ]:
Vx=.5*x.^2;
Vk=.5*k.^2

Uxh=exp.(-Vx*dt/2);
Ux=exp.(-Vx*dt);
Uf=exp.(-Vk*dt);
"potentials and evolvers defined"

In [ ]:
ϕ(x)=π^(-.25)*exp(-x.^2/2)
ϕ1(x)=sqrt(2)*π^(-.25)*exp(-x.^2/2)*x
ϕ2(x)=1/sqrt(2)*π^(-.25)*exp(-x.^2/2)*(2*x^2-1)
ϕ4(x)=1/(2*sqrt(6)*π^(.25))*exp(-x.^2/2)*(4*x^4-12*x^2+3)

In [ ]:
Ψ0=(1.0/(2*xmax))*ones(Complex{Float64},N);
Ψtrue=(1+0*im).*ϕ.(x);
Ψ1=(1+0*im).*ϕ1.(x);
Ψ2=(1+0*im).*ϕ2.(x);
Ψ4=(1+0*im).*ϕ4.(x);

plot(x,abs.(Ψ0),label="Starting Wavefunction")
plot!(x,abs.(Ψtrue),label="Analytic Ground State")
plot!(x,abs.(Ψ1),label="First Excited")
plot!(x,abs.(Ψ2),label="Second Excited")


plot!(xlabel="x",ylabel="Absolute Value",title="Wavefunctions")

In [ ]:
nmeas

###  Planning and Initializing

In [ ]:
ft=plan_fft(Ψ0);
Ψf=ft*Ψ0;
ift=plan_ifft(Ψf);

In [ ]:
nmeas=1
lmeas=floor(Int, nt/nmeas)

Ex=zeros(Float64,lmeas);
Ek=zeros(Float64,lmeas);
E=zeros(Float64,lmeas);
E2=zeros(Float64,lmeas);

c=zeros(Float64,lmeas);
c1=zeros(Float64,lmeas);
c2=zeros(Float64,lmeas);
c4=zeros(Float64,lmeas);

## The Code Body

Now we finally get to run the body of the code.  If you look back over the Split Operator Method Post, you will see almost identical code, as I am doing virtually the same thing.  I am only using different evolution operators and measuring different quantities.

In [ ]:
Ψ0=(1.0/(2*xmax))*ones(Complex{Float64},N);
Ψ=copy(Ψ0);
jj=1

# The operators we have to start off with
Ψ=Ψ.*Uxh

Psif=(ft*Ψ)*dx
Psif=Psif.*Uf    
Ψ=(ift*Psif)*(dk/2π)

for ii in 1:nt
    
    Ψ=Ψ.*Ux
    
    
    Psif=(ft*Ψ)*dx
    Psif=Psif.*Uf 
    
    Ψ=(ift*Psif)*(dk/2π)   
    nn=sum(conj(Ψ).*Ψ)*dx
    Ψ=1/sqrt(nn)*Ψ
    
    if ii%nmeas == 0
        # Every time we measure, we have to finish with a U_x half time step
        Ψt=Ψ.*Uxh
        Ψft=(ft*Ψt)*dx
    
        Ex[jj]=real(sum(conj(Ψt).*Vx.*Ψt))*dx
        
        Ek[jj]=real(sum(conj(Ψft).*Vk.*Ψft))*(dk/2π)
    
        E[jj]=Ex[jj]+Ek[jj]
        
        c[jj]=abs(sum( conj(Ψt).* Ψtrue)) *dx
        c1[jj]=abs(sum( conj(Ψt).* Ψ1)) *dx
        c2[jj]=abs(sum( conj(Ψt).* Ψ2)) *dx
        c4[jj]=abs(sum( conj(Ψt).* Ψ4)) *dx
        
        jj+=1
    end
end    


Ψ=Ψ.*Uxh;

## Results

In this next section, we examine the results that we calculated above, making sure they agree with our analytical predictions.  

The first two pictures show how the final wavefunction after imaginary time evolution agrees with the ground state for the Quantum Harmonic Osscilator, both in position and momentum space.  

In [ ]:
plot(x,abs.(Ψ),linewidth=10, label="Final")
plot!(x,abs.(Ψtrue), linewidth=5,label="Analytic")

plot!(xlabel="x",ylabel="Absolute Value",title="Wavefunction")

In [ ]:
plot(fftshift(k),abs.(fftshift(fΨ)),linewidth=10,label="Final")
plot!(fftshift(k),abs.(fftshift(fΨtrue)), linewidth=5,label="Analytic")

plot!(xlabel="k",ylabel="Absolute Value",title="Fourier Transform of Wavefunction")

The Energy of the system converges to $1/2 \hbar \omega$, which in this case is just 1/2.  The potential and kinetic energy each contribute to half of the total energy.

In [ ]:
plot(E,label=" Total Energy")
plot!(Ex,label="Potential")
plot!(Ek,label="Kinetic")
hline!([1/2],label="1/2")

plot!(xlabel="Steps",ylabel="Energy",title="Convergence of Energy")

Here we look at how the state converges to the ground state by dissipating out higher energy states.  

We start with a even ground state, so any odd eigenstates, like the first excited state, will have no occupation starting out.  Thus, the odd eigenstates will never have any occupation.  This highlights the importance of a good first guess, though numerical error might save the day if the ground state wasn't present in our starting state.

The higher the energy of the state, the faster it will decay.  We even see that the Second Excited State increases in occupation for a short while in the beginning as the higher energy states rapidly disappear.  

In [ ]:
plot(c,label="Ground State")
plot!(c1,label="First Excited")
plot!(c2,label="Second Excited")
plot!(c4,label="Fourth Excited")

plot!(xlabel="Steps",ylabel="Occupation of State",title="Occupation of Eigenstates")